In [ ]:
import csv


def get_lists_platforms_and_series():

    series = []
    platforms = []

    with open('geo_samples.tsv','r') as tsvin:
        tsvin = csv.reader(tsvin, delimiter='\t')
        indexes = []
        for (i, row) in enumerate(tsvin):
            if i == 0:
                indexes = [row.index(j) for j in ['Series', 'Taxonomy', 'Platform']]
            
            if row[indexes[1]].lower() in {"homo sapiens", "mus musculus", "rattus norvegicus"}:
                print(i)
                ser = row[indexes[0]].split(";")
                for s in ser:
                    if s not in series: 
                        series.append(s)
                if row[indexes[2]] not in platforms:
                    platforms.append(row[indexes[2]])
    
    write_to_file(series, "list_series_for_download.txt")
    write_to_file(platforms, "list_platforms_for_download.txt")
    
    return series, platforms
            

def write_to_file(list_for_download, file_name): 
    text_file = open(file_name, "w")
    
    for item in list_for_download:
        text_file.write("%s\n" % item)
    text_file.close()

In [ ]:
series, platforms = get_lists_platforms_and_series()
print("________________________________________________________________-")
print(len(series))
print(len(platforms))

In [ ]:
from ftplib import FTP
from tqdm import trange
import csv


def get_series_from_web():
    series_organism = read_file("list_series_for_download.txt")
    
    ftp = FTP('ftp.ncbi.nlm.nih.gov')
    ftp.login()

    ftp.cwd('/geo/series/')
    series_list = ftp.nlst()

    matrices = {}

    for i in trange(0,len(series_list),1):
        print('current serie: ', series_list[i], ' with i: ', i)
        
        ftp.cwd('/geo/series/' + series_list[i] + '/')

        datasets_list = ftp.nlst()
        
        for j in range(0,len(datasets_list),1):
            path = series_list[i] + "/" + datasets_list[j] + "/matrix/"
            if datasets_list[j] in series_organism:
                matrices[datasets_list[j]] = {'path': path}
                
        print("_____________________________________________________________________")
    
    return matrices


matrices = get_series_from_web()
write_dict_to_file(matrices, "list_series.csv")

In [ ]:
from ftplib import FTP
from tqdm import trange
import csv


def get_platforms_from_web(): 
    platforms_organism = read_file("list_platforms_for_download.txt")

    ftp = FTP('ftp.ncbi.nlm.nih.gov')
    ftp.login()

    ftp.cwd('/geo/platforms/')
    platforms_list = ftp.nlst()

    annot = {}

    for i in trange(0,len(platforms_list),1):
        print('current platform: ', platforms_list[i], ' with i: ', i)
        
        ftp.cwd('/geo/platforms/' + platforms_list[i] + '/')
        gpl_list = ftp.nlst()

        for j in range(0,len(gpl_list),1):           
            path = platforms_list[i] + "/" + gpl_list[j] + "/annot/"
            if gpl_list[j] in platforms_organism:
                annot[gpl_list[j]] = {'path': path}
           
        print("__________________________________________________________________")
    
    return annot


annot = get_platforms_from_web()
write_dict_to_file(annot, "list_platforms.csv")

In [ ]:
def write_dict_to_file(dict_for_download, file_name):
        
    with open(file_name, 'w') as csvout:
        csvout = csv.writer(csvout)
        rows = []
        rows.append(['key', 'path'])
        for key, value in dict_for_download.items():
            rows.append([key, value['path']])  
        csvout.writerows(rows)


def read_file(file_name):
    lines = [line.rstrip('\n') for line in open(file_name)]
    return lines

In [ ]:
with open('list_series.csv','r') as tsvin:
    tsvin = csv.reader(tsvin, delimiter=',')
    indexes = []
    for (i, row) in enumerate(tsvin):
        if i == 0:
            indexes = [row.index(j) for j in ['key', 'path']]
            print(indexes[1])
            
        print(row[indexes[0]], ' ', row[indexes[1]])

In [ ]:
import subprocess
from ftplib import FTP
import csv
import os


def download_data(file_name, kind_data): 
    with open(file_name,'r') as tsvin:
        tsvin = csv.reader(tsvin, delimiter=',')
        indexes = []
        for (i, row) in enumerate(tsvin):
            if i == 0:
                indexes = [row.index(j) for j in ['key', 'path']]
                
            try:
                directory = "../geo/" + kind_data + "/" + row[indexes[1]]

                ftp = FTP('ftp.ncbi.nlm.nih.gov')
                ftp.login()
                ftp.cwd('/geo/' + kind_data + '/' + row[indexes[1]])
                data_list = ftp.nlst()

                for data in data_list:
                    download = 'https://ftp.ncbi.nlm.nih.gov/geo/' + kind_data + '/' + \
                                row[indexes[1]] + data

                    subprocess.call(["mkdir", "-p", directory])

                    if data not in os.listdir(directory):
                        subprocess.call(["wget", "-c", "-P", directory, download])
            except:
                pass

            print('current', kind_data, ': ', row[indexes[0]], 'path: ', \
                                          row[indexes[1]], ' with i: ', i)

        
        
download_data("list_platforms.csv", 'platforms')
#download_data("list_series.csv", 'series')